In [2]:
import pandas as pd

dataset = pd.read_excel(r"G:\Gayathri\Name Segmentation\gpt_src.xlsx")

In [3]:
dataset.head()

,Name,Revsd Name
0,Ananya Cosmetic,Ananya Cosmetics
1,Aneja Medic,Aneja Medical Store
2,"Apna Bazar 44 ,Jain Interprises",Apna Bazaar 44 - Jain Enterprises
3,APNA DEPARTMENT STORE,Apna Departmental Store
4,Apna Dhaba(Vikaspuri),Apna Dhaba


In [4]:
#Training Data format
def convert_to_gpt35_format(dataset):
    fine_tuning_data = []
    for row in dataset.values:
        fine_tuning_data.append({
            "messages": [
                {"role": "system", "content": "Clean or correct the spelling for the store name and give the store name alone"},
                {"role": "user", "content": row[0]},
                {"role": "assistant", "content" : row[1]}
            ]
        })
    return fine_tuning_data

converted_data = convert_to_gpt35_format(dataset)

In [5]:
converted_data[5:15]

[{'messages': [{'role': 'system',
    'content': 'Clean or correct the spelling for the store name and give the store name alone'},
   {'role': 'user', 'content': 'Apna Drug Bazar'},
   {'role': 'assistant', 'content': 'Apna Drug Bazaar'}]},
 {'messages': [{'role': 'system',
    'content': 'Clean or correct the spelling for the store name and give the store name alone'},
   {'role': 'user', 'content': 'Apna Dryfruit Store'},
   {'role': 'assistant', 'content': 'Apna Dry fruits Store'}]},
 {'messages': [{'role': 'system',
    'content': 'Clean or correct the spelling for the store name and give the store name alone'},
   {'role': 'user', 'content': 'APNA DUKAAN'},
   {'role': 'assistant', 'content': 'Apna Dukan'}]},
 {'messages': [{'role': 'system',
    'content': 'Clean or correct the spelling for the store name and give the store name alone'},
   {'role': 'user', 'content': 'Shri Samarth Super Market'},
   {'role': 'assistant', 'content': 'Shri Samarth Supermarket'}]},
 {'messages': [

In [6]:
import json
import openai
openai.api_key = 'sk-XYZ'

def write_to_jsonl(data, file_path):
    with open(file_path, 'w') as file:
        for entry in data:
            json.dump(entry, file)
            file.write('\n')

file_name = r"G:\Gayathri\Self_projects\Name Segmentaion\Training_data.jsonl"

write_to_jsonl(converted_data, file_name)

training_file = openai.files.create(
    file=open(file_name, "rb"), purpose="fine-tune"
)

In [7]:
response = openai.fine_tuning.jobs.create(
    training_file=training_file.id,
    model="gpt-3.5-turbo",
)
response

FineTuningJob(id='ftjob-i8uvlMpu3oQNPmDjmB894Qo9', created_at=1716790964, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-6HGmMlbdiBqTAxRnh1unJ3dC', result_files=[], seed=1286120256, status='validating_files', trained_tokens=None, training_file='file-he1jtzCurErOEUgAAZKUXwuR', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

In [8]:
fine_tune_events = openai.fine_tuning.jobs.list_events(response.id)
fine_tune_events

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-Nzz473qIYNL47JbZ9HYIp3Dr', created_at=1716790964, level='info', message='Validating training file: file-he1jtzCurErOEUgAAZKUXwuR', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-LdESRprzv4IY582kRfB24UJr', created_at=1716790964, level='info', message='Created fine-tuning job: ftjob-i8uvlMpu3oQNPmDjmB894Qo9', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [10]:
openai.fine_tuning.jobs.list(limit=10)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-i8uvlMpu3oQNPmDjmB894Qo9', created_at=1716790964, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-6HGmMlbdiBqTAxRnh1unJ3dC', result_files=[], seed=1286120256, status='running', trained_tokens=None, training_file='file-he1jtzCurErOEUgAAZKUXwuR', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None), FineTuningJob(id='ftjob-KrgY2llNFEIGH1AWKruV5vJX', created_at=1716556647, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:personal::9SPBqDmz', finished_at=1716557041, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-6HGmMlbdiBqTAxRnh1unJ3dC', re

In [12]:
retrieve_response = openai.fine_tuning.jobs.retrieve(response.id)
retrieve_response

FineTuningJob(id='ftjob-i8uvlMpu3oQNPmDjmB894Qo9', created_at=1716790964, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-6HGmMlbdiBqTAxRnh1unJ3dC', result_files=[], seed=1286120256, status='running', trained_tokens=None, training_file='file-he1jtzCurErOEUgAAZKUXwuR', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

In [13]:
#FINETUNED MODEL RECEIVED VIA REGISTERED MAIL ADDRESS.

In [14]:
#Finetuned model received from mail is "ft:gpt-3.5-turbo-0125:personal::XYZ"

In [30]:
#Genral GPT MODEL
import openai
openai.api_key = 'sk-XYZ'
# Without assistant content
system_content = "Clean or correct the spelling for the store name and give the store name alone"
prompt1 = "Kvein GS"

response = openai.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[
        {"role": "system", "content": system_content},
        {"role": "user", "content": prompt1},
    ],temperature=0, max_tokens=50)

parse_out = response.choices[0].message.content
print("GPT MODEL:",parse_out)

GPT MODEL: Kevin GS


In [31]:
#FINE TUNED MODEL GPT MODEL
import openai
openai.api_key = 'sk-XYZ'
# Without assistant content
system_content = "Clean or correct the spelling for the store name and give the store name alone"
prompt1 = "Kvein GS"

response = openai.chat.completions.create(
     
    model='ft:gpt-3.5-turbo-0125:personal::XYZ',
    messages=[
        {"role": "system", "content": system_content},
        {"role": "user", "content": prompt1},
    ],temperature=0, max_tokens=50)

parse_out = response.choices[0].message.content
print("Fine Tuned Model:",parse_out)


Fine Tuned Model: Kevin General Store


In [29]:
#FINE TUNED MODEL GPT MODEL
import openai
openai.api_key = 'sk-XYZ'
system_content = "Clean or correct the spelling for the store name and give the store name alone"
prompt1 = "OYO Santi Inn Near Dilli Haat INA"
response = openai.chat.completions.create(
     
    model='ft:gpt-3.5-turbo-0125:personal::XYZ',
    messages=[
        {"role": "system", "content": system_content},
        {"role": "user", "content": prompt1},
    ],temperature=0, max_tokens=50)

parse_out = response.choices[0].message.content
print("Fine Tuned Model:",parse_out)


Fine Tuned Model: Oyo Santi Inn


In [28]:
#Genral GPT MODEL
import openai
openai.api_key = 'sk-XYZ'
system_content = "Clean or correct the spelling for the store name and give the store name alone"
prompt1 = "OYO Santi Inn Near Dilli Haat INA"
response = openai.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[
        {"role": "system", "content": system_content},
        {"role": "user", "content": prompt1},
    ],temperature=0, max_tokens=50)

parse_out = response.choices[0].message.content
print("GPT MODEL:",parse_out)

GPT MODEL: Santi Inn




In [33]:
#FINE TUNED MODEL GPT MODEL
import openai
openai.api_key = 'sk-XYZ'
system_content = "Clean or correct the spelling for the store name and give the store name alone"
prompt1 = "Mundar Saundrya Prasadhan Kendra, Neha General Store"
response = openai.chat.completions.create(
     
    model='ft:gpt-3.5-turbo-0125:personal::XYZ',
    messages=[
        {"role": "system", "content": system_content},
        {"role": "user", "content": prompt1},
    ],temperature=0, max_tokens=50)

parse_out = response.choices[0].message.content
print("Fine Tuned Model:",parse_out)


Fine Tuned Model: Neha General Store


In [32]:
#Genral GPT MODEL
import openai
openai.api_key = 'sk-XYZ'
system_content = "Clean or correct the spelling for the store name and give the store name alone"
prompt1 = "Mundar Saundrya Prasadhan Kendra, Neha General Store"
response = openai.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[
        {"role": "system", "content": system_content},
        {"role": "user", "content": prompt1},
    ],temperature=0, max_tokens=50)

parse_out = response.choices[0].message.content
print("GPT MODEL:",parse_out)

GPT MODEL: Mundar Saundrya Prasadhan Kendra

Neha General Store
